In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import StratifiedKFold

In [5]:
SEED = 42 
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [6]:
dir_path = "../process/"
# Cargar el archivo limpio
df = pd.read_csv(f"{dir_path}train_data.csv")
print(set(df["source"]))
# Verificar estructura y primeras filas
print(df.shape)
print(df.columns)
df.head()

{'Hadoop', 'Apache', 'BGL', 'Android', 'Zookeeper'}
(9090, 6)
Index(['Content', 'Level', 'datetime', 'content_length', 'source',
       'Clean_Content'],
      dtype='object')


,Content,Level,datetime,content_length,source,Clean_Content
0,setLightsOn(true),Informativo,2024-03-17 16:14:21.261,2,Android,setlightson true
1,suspendAutohide,Informativo,2024-03-17 16:13:46.350,1,Android,suspendautohide
2,setLightsOn(true),Informativo,2024-03-17 16:14:01.964,2,Android,setlightson true
3,setSystemUiVisibility vis=0 mask=1 oldVal=4000...,Informativo,2024-03-17 16:14:09.564,12,Android,setsystemuivisibility vis mask oldval newval d...
4,userActivityNoUpdateLocked: eventTime=26199130...,Informativo,2024-03-17 16:16:06.670,5,Android,useractivitynoupdatelocked eventtime event fla...


In [10]:
print("Distribución original:")
print(df['Level'].value_counts())

Distribución original:
Level
Informativo    6038
Advertencia    2253
Error           799
Name: count, dtype: int64


In [12]:
min_count = df['Level'].value_counts().min()

balanced_df = (
    df.groupby('Level')
    .apply(lambda x: x.sample(n=min_count, random_state=42))
    .reset_index(drop=True)
)
min_count

/tmp/ipykernel_12731/503412410.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_count, random_state=42))


799

In [21]:
# Crear folds balanceados
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['Level'])):
    train_df = df.iloc[train_idx].reset_index(drop=True)
    val_df = df.iloc[val_idx].reset_index(drop=True)
    
    min_count_train = train_df['Level'].value_counts().min()
    min_count_val = val_df['Level'].value_counts().min()

    # Balancear ambos (train y val)
    train_df_balanced = (
        train_df.groupby('Level')
        .apply(lambda x: x.sample(n=min_count_train, random_state=SEED + fold))  # distinta semilla por fold
        .reset_index(drop=True)
    )

    val_df_balanced = (
        val_df.groupby('Level')
        .apply(lambda x: x.sample(n=min_count_val, random_state=SEED + fold))
        .reset_index(drop=True)
    )

    # Guardar archivos
    train_df_balanced.to_csv(os.path.join(folds_dir, f"train_fold_{fold}.csv"), index=False)
    val_df_balanced.to_csv(os.path.join(folds_dir, f"val_fold_{fold}.csv"), index=False)

    # Mostrar distribución
    print(f"\n Fold {fold} guardado:")
    print("Train:", train_df_balanced['Level'].value_counts().to_dict(), train_df_balanced.shape[0])
    print("Val:  ", val_df_balanced['Level'].value_counts().to_dict(), val_df_balanced.shape[0])


 Fold 0 guardado:
Train: {'Advertencia': 720, 'Error': 720, 'Informativo': 720} 2160
Val:   {'Advertencia': 79, 'Error': 79, 'Informativo': 79} 237

 Fold 1 guardado:
Train: {'Advertencia': 719, 'Error': 719, 'Informativo': 719} 2157
Val:   {'Advertencia': 80, 'Error': 80, 'Informativo': 80} 240

 Fold 2 guardado:
Train: {'Advertencia': 719, 'Error': 719, 'Informativo': 719} 2157
Val:   {'Advertencia': 80, 'Error': 80, 'Informativo': 80} 240

 Fold 3 guardado:
Train: {'Advertencia': 719, 'Error': 719, 'Informativo': 719} 2157
Val:   {'Advertencia': 80, 'Error': 80, 'Informativo': 80} 240

 Fold 4 guardado:
Train: {'Advertencia': 719, 'Error': 719, 'Informativo': 719} 2157
Val:   {'Advertencia': 80, 'Error': 80, 'Informativo': 80} 240

 Fold 5 guardado:
Train: {'Advertencia': 719, 'Error': 719, 'Informativo': 719} 2157
Val:   {'Advertencia': 80, 'Error': 80, 'Informativo': 80} 240

 Fold 6 guardado:
Train: {'Advertencia': 719, 'Error': 719, 'Informativo': 719} 2157
Val:   {'Advertencia

/tmp/ipykernel_12731/1953447422.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_count_train, random_state=SEED + fold))  # distinta semilla por fold
/tmp/ipykernel_12731/1953447422.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_count_val, random_state=SEED + fold))
/tmp/ipykernel_12731/1953447422.py:12: DeprecationWarning: DataFrameG